In [58]:
import torch
import torch.nn as nn
import math, copy, time
import pandas as pd
import altair as alt

# Encoder-Decoder Architecture
A standard encoder decoder architecture as set out in the paper "Attention is all you need" at https://arxiv.org/pdf/1706.03762.pdf
Note: At each step the model is auto-regressive (consuming prev generated symbol as new input)
Args: 
- encoder: (nn.Module)
    - neural net that takes in a sequence of symbol representations `(x1, x2, .... xn)` and outputs a continuous representation `z = (z1, z2, .... zn)`
    - token embeddings form the symbol sequence, and single vector which is analogous to a one hot vector of the composite sentence is the continuous representation
    - takes input the `source embedding` and the `mask` as the embeddings are padded to a constant size
- decoder: (nn.Module)
    - takes the continuous representation `z` and generates an output sequence `(y1, y2, ... ym)`
- generator: (nn.Module)
    - takes the probability distribution outputted by the decoder and generates the text token
    
<div>
<img src="img/enc-dec.svg" width="200"/>
</div>


In [36]:
# Hyperparams
D_MODEL = 16       # number of dimensions handled by the network
N = 6              # number of encoders in the encoder stack

In [37]:
class EncoderDecoder(nn.Module):
    """
    Base class that implements a black-box encoder decoder architecture as set out in the transformers paper.
    Translation use case
    """
    
    def __init__(self, encoder, decoder, src_embeddings, tgt_embeddings, generator):
        super().__init__()
        self.encoder = encoder 
        self.decoder = decoder 
        self.generator = generator
        self.src_embeddings = src_embeddings        # embeddings table for input tokens
        self.tgt_embeddings = tgt_embeddings        # embeddings table for target tokens
        
    def forward(self, x):
        return self.decode(self.encode(x,...),...)
        
    def encode(self):
        return self.encoder(...)
    
    def decode(self):
        return self.decoder(...)

class Generator(nn.Module):
    """
    Does a linear + softmax operation to output the tokens
    """
    
    #! see if you can auto-initialize this with hyperparams
    def __init__(self, model_dims, vocab_size):
        super().__init__()
        
        self.linear = nn.Linear(model_dims, vocab_size)
        self.softmax = nn.LogSoftmax(dim=-1)
    
    def forward(self, x):
        return self.softmax(self.layer(x))
        

<div>
<img src="img/encoder_1.1.svg" width="300"/>
</div>

In [40]:
def clone(module_to_clone, num_of_clones):
    assert isinstance(num_of_clones, numbers.Integral)
    return nn.ModuleList([copy.deepcopy(module_to_clone) for _ in range(num_of_clones)])

class Encoder(nn.Module):
    " core encoder which is a stack of 6 individual encoders in sequence"
    
    def __init__(self, encoder_layer, N):
        super().__init__()
        self.encoder_stack = clone(encoder_layer, N)
        
    def forward(self, x):
        for encoder in encoder_stack:
            x = encoder(x)
        return x

We employ a residual connection (cite) around each of the two sub-layers, followed by layer normalization (cite).


<div>
    <img src="img/enc-sublayers.svg" width='400'/>
</div>

`LayerNorm` and `BatchNorm` are similar but different in where they apply their normalization. In Batch norm, the `median` and the `standard deviation` are applied across the incoming batch, whereas in LayerNorm, the same statistics are calculated across the dimensions of the input to the layer. 
- Hence BatchNorm is across the batch 
- LayerNorm is across each input. 

This is a good article explaining the differences:
https://www.pinecone.io/learn/batch-layer-normalization/

To address this, batch normalization introduces two parameters: a scaling factor gamma (γ) and an offset beta (β). These are learnable parameters, so if the fluctuation in input distribution is necessary for the neural network to learn a certain class better, then the network learns the optimal values of gamma and beta for each mini-batch.

$$ 
\begin{align}
\mu_l = \frac{1}{d}\sum_{i=1}^{d}x_i \text{}\text{ } (1)\\ \sigma_l^2 = \frac{1}{d}\sum_{i=1}^{d}(x_i - \mu_l)^2 \text{}\text{ } (2)\\ \hat{x_i} = \frac{x_i - \mu_l}{\sqrt{\sigma_l^2}} \text{}\text{ } (3)\\ or\text{ }\hat{x_i} = \frac{x_i - \mu_l}{\sqrt{\sigma_l^2 + \epsilon}} \text{}\text{ } (3) \\ Adding\text{ }\epsilon\text{ }helps\text{ }when\text{ }\sigma_l^2\text{ }is\text{ }small\\ y_i = \mathcal{LN}(x_i) = \gamma.x_i + \beta \text{}\text{ }(4)
\end{align}
 $$

<div>
    <img src="https://d33wubrfki0l68.cloudfront.net/5863322b42dcdf4b45ffef4de43f6ef0385db477/e6251/images/batch-normalization-example.png" width='400'/>
    <img src="https://d33wubrfki0l68.cloudfront.net/c8f1f7a886548f82234f8a3b06faeecfbb88c657/42d49/images/layer-normalization.png" width='400'/>
</div>
    


In [39]:
class LayerNorm(nn.Module):
    """
    LayerNorm module
    """
    
    def __init__(self, features, eps=1e-6):
        super().__init__()
        self.eps = eps         # handle division by zero
        self.gamma = nn.Parameter(torch.ones(features))        # scaling factor that the the network learns
        self.beta = nn.Parameter(torch.zeros(features))        # offset factor that the the network learns
    
    def forward(self, x):
        std = x.std(dim=-1, keepdim=True)
        mean = x.mean(dim=-1, keepdim=True)
        return self.gamma * (x-mean)/(std + eps) + self.beta

That is, the output of each sub-layer is $\mathrm{LayerNorm}(x + \mathrm{Sublayer}(x))$, where $\mathrm{Sublayer}(x)$ is the function implemented by the sub-layer itself. We apply dropout (cite) to the output of each sub-layer, before it is added to the sub-layer input and normalized.

To facilitate these residual connections, all sub-layers in the model, as well as the embedding layers, produce outputs of dimension d_{\text{model}}=512d 
model =512.

In [ ]:
class Sublayer(nn.Module):
    
    def __init__(self, size, dropout):
        super().__init__()
        self.layernorm = LayerNorm(size)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x, sublayer_protagonist):
#       return self.layernorm(x + self.dropout(sublayer_protagonist(x)))
#       return x + self.dropout(sublayer_protagonist(x))
        return x + self.dropout(sublayer_protagonist(self.norm(x)))
        

Each layer has two sub-layers. The first is a multi-head self-attention mechanism, and the second is a simple, position-wise fully connected feed-forward network.

In [ ]:
class EncoderLayer(nn.Module):
    """
    Collection of two sublayers that make up a single Encoder layer
    """
    
    def __init__(self, size, multihead_self_attention, feed_forward, dropout):
        super().__init__()
        self.attention = multihead_self_attention
        self.feed_forward = feed_forward 
        self.sublayers = clone(Sublayer(size, dropout), 2)
        self.size = size 
    
    def forward(self, x):
        # sublayer 1's output 
        x = self.sublayers[0](x, self.attention(...))
        return self.sublyers[1](x, self.feed_forward)

# Decoder
The decoder is also composed of a stack of N=6 identical layers.

In [41]:

class Decoder(nn.Module):
    " core decoder which is a stack of 6 individual decoders in sequence"
    
    def __init__(self, decoder_layer, N):
        super().__init__()
        self.decoder_stack = clone(decoder_layer, N)
        
    def forward(self, x):
        for decoder in decoder_stack:
            x = decoder(x)
        return x

In addition to the two sub-layers in each encoder layer, the decoder inserts a third sub-layer, which performs multi-head attention over the output of the encoder stack. Similar to the encoder, we employ residual connections around each of the sub-layers, followed by layer normalization.

In [ ]:
class DecoderLayer(nn.Module):
    
    """
    This defines each layer of the decoder that is composed of 3 sublayers each
    """
    
    def __init__(
        self, 
        size, 
        masked_multi_head_attention, 
        multi_head_attention, 
        feed_forward, 
        dropout
    ):
        
        
        super().__init__()
        self.masked_attention = masked_multi_head_attention
        self.attention = multi_head_attention
        self.feed_forward = feed_forward
        self.sublayers = clone(Sublayer(size, dropout), 3)
    
    def forward(self, x):
        x = self.sublayers[0](x, self.masked_attention(...))
        x = self.sublayers[1](x, self.attention(...))
        x = self.sublayers[2](x, self.feed_forward(...))
        return x

We also modify the self-attention sub-layer in the decoder stack to prevent positions from attending to subsequent positions. This masking, combined with fact that the output embeddings are offset by one position, ensures that the predictions for position `i` can depend only on the known outputs at positions less than `i`.

In [44]:
def forward_mask(size):
    """
    Mask to prevent current word being affected by words after it. Only past words affect.
    
    If input is:
    [*,*,*,*]
    [*,*,*,*]
    [*,*,*,*]
    [*,*,*,*]
    
    return:
    [False, False, False, False]
    [ True, False, False, False]
    [ True,  True, False, False]
    [ True,  True,  True, False]
    """
    attn_shape = (1, size, size)
    mask = torch.triu(torch.ones(attn_shape), diagonal=1).type(torch.uint8)
    return mask==0


In [72]:
# a visualization of a sample mask
def show_sample_mask(size):
    LS_data = pd.concat(
    [
            pd.DataFrame(
                {
                    "Forward Mask": forward_mask(size)[0][x, y].flatten(),
                    "Window": y,
                    "Masking": x,
                }
            )
            for y in range(size)
            for x in range(size)
        ]
    )
    
    return (
        alt.Chart(LS_data)
        .mark_rect()
        .properties(height=250, width=250)
        .encode(
        alt.X("Window:O"),
        alt.Y("Masking:O"),
        alt.Color("Forward Mask:Q", scale=alt.Scale(scheme="viridis"))
        )
        .interactive()
        
        )


show_sample_mask(20)

alt.Chart(...)

Attention
An attention function can be described as mapping a query and a set of key-value pairs to an output, where the query, keys, values, and output are all vectors. The output is computed as a weighted sum of the values, where the weight assigned to each value is computed by a compatibility function of the query with the corresponding key.

We call our particular attention “Scaled Dot-Product Attention”. The input consists of queries and keys of dimension d_kd 
k
​
 , and values of dimension d_vd 
v
​
 . We compute the dot products of the query with all keys, divide each by \sqrt{d_k} 
d 
k
​
 
​
 , and apply a softmax function to obtain the weights on the values.



In [43]:
t = torch.ones(5,5)
t_zeros = torch.triu(t, diagonal=1)
t_zeros == 0

tensor([[ True, False, False, False, False],
        [ True,  True, False, False, False],
        [ True,  True,  True, False, False],
        [ True,  True,  True,  True, False],
        [ True,  True,  True,  True,  True]])